In [14]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import gradio as gr
from datetime import timedelta
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [15]:

#functions 
def analyze_tweets(tweets):
    #delimiter 
    tweet_list = tweets.split('|||')
       
    compound_scores = [sid.polarity_scores(tweet.strip())['compound'] for tweet in tweet_list if tweet.strip()]
       
    if compound_scores:
        average_sentiment = sum(compound_scores) / len(compound_scores)
    else:
        return "No  tweets entered."
    
    return f"The average compound sentiment score for the entered tweets is: {average_sentiment}"


def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(seq_length, len(data)):
        past_60_days = data[i-seq_length:i, 0]
        current_features = data[i, [1, 2, 3]]
        sequence = np.concatenate((past_60_days, current_features), axis=0)
        sequences.append(sequence)
        labels.append(data[i, 0])
    return np.array(sequences), np.array(labels)

def fetch_recent_stock_data(stock_symbol, period='3mo'):
    stock_data = yf.download(stock_symbol, period=period)
    if stock_data.empty:
        raise ValueError(f"No data found for {stock_symbol}.")
    return stock_data

#close prediction and caculate dates
def predict_next_day_close_yfinance(stock_symbol, sentiment_score, number_of_tweets):
    stock_data = fetch_recent_stock_data(stock_symbol)
    if len(stock_data) < 60:
        raise ValueError(f"Not enough data available for {stock_symbol}.")
    
    past_60_days_close = stock_data['Close'][-60:].values
    most_recent_data = stock_data.iloc[-1]
    open_price = most_recent_data['Open']
    
    past_60_days_scaled = scaler.transform(np.column_stack((past_60_days_close, np.zeros((60, 3)))))[0:60, 0]
    current_features_scaled = scaler.transform([[0, sentiment_score, number_of_tweets, open_price]])[0, 1:]
    
    input_sequence = np.concatenate((past_60_days_scaled, current_features_scaled), axis=0)
    input_sequence = np.reshape(input_sequence, (1, 1, len(input_sequence)))
    
    predicted_price_scaled = model.predict(input_sequence)
    predicted_price = scaler.inverse_transform([[predicted_price_scaled[0][0], 0, 0, 0]])[0][0]

    #calculate the next trading day
    latest_date = stock_data.index[-1]
    next_prediction_date = latest_date + timedelta(days=1)
    
    return predicted_price, next_prediction_date

#train model on selected stock
def train_model(stock_symbol):
    stock_specific_data = merged_data[merged_data['Stock Name'] == stock_symbol].sort_values(by='Date')
    
    training_columns = ['Close', 'Average_Sentiment', 'Number_of_Tweets', 'Open']
    training_data = stock_specific_data[training_columns].dropna()
    
    #scaling
    global scaler  # Make scaler global to use it in other functions
    scaler = MinMaxScaler()
    scaled_training_data = scaler.fit_transform(training_data)
    
    #sequences
    sequence_length = 60
    X, y = create_sequences(scaled_training_data, sequence_length)
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    #LSTM Model
    global model  #model global to use it in other functions
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=1, epochs=10, validation_data=(x_test, y_test))
    
    #evaluate the model
    evaluation_results = evaluate_model(model, x_test, y_test, scaler)
    
    return f"Model trained on {stock_symbol}.\nEvaluation results:\nMSE: {evaluation_results['MSE']}\nMAE: {evaluation_results['MAE']}\nR²: {evaluation_results['R2']}"

def evaluate_model(model, x_test, y_test, scaler):
    #predict on the test data
    y_pred_scaled = model.predict(x_test)
    
    #reshape y_test to match y_pred shape
    y_test = y_test.reshape(-1, 1)
    
    #inverse scale
    y_pred = scaler.inverse_transform(
        np.concatenate([y_pred_scaled, np.zeros((y_pred_scaled.shape[0], 3))], axis=1)
    )[:, 0]
    y_test_original = scaler.inverse_transform(
        np.concatenate([y_test, np.zeros((y_test.shape[0], 3))], axis=1)
    )[:, 0]
    
    #eval metrics
    mse = mean_squared_error(y_test_original, y_pred)
    mae = mean_absolute_error(y_test_original, y_pred)
    r2 = r2_score(y_test_original, y_pred)
    
    return {'MSE': mse, 'MAE': mae, 'R2': r2}

def predict_stock_price(stock_symbol, sentiment_score, number_of_tweets):
    
    train_message = train_model(stock_symbol)
    
    
    predicted_price, prediction_date = predict_next_day_close_yfinance(stock_symbol, sentiment_score, number_of_tweets)
    
    return f"{train_message}\nThe predicted closing price for {stock_symbol} on {prediction_date.date()} is: {predicted_price}"

In [11]:
merged_data['Stock Name'].unique()

array(['AAPL', 'AMD', 'AMZN', 'DIS', 'GOOG', 'META', 'MSFT', 'NFLX',
       'NIO', 'PG', 'TSLA', 'TSM', 'XPEV', 'CRM', 'ZS', 'ENPH', 'PYPL',
       'BA', 'COST', 'KO', 'BX', 'F', 'INTC', 'NOC', 'VZ'], dtype=object)

In [16]:
#file 
file_path = 'merged_sentiment_stock_data.csv'
merged_data = pd.read_csv(file_path)

In [ ]:
#stock tab
stock_price_tab = gr.Interface(
    fn=predict_stock_price,
    inputs=[
        gr.Dropdown(label="Stock Symbol", choices=['AAPL', 'AMD', 'AMZN', 'DIS', 'GOOG', 'META', 'MSFT', 'NFLX',
       'NIO', 'PG', 'TSLA', 'TSM', 'XPEV', 'CRM', 'ZS', 'ENPH', 'PYPL',
       'BA', 'COST', 'KO', 'BX', 'F', 'INTC', 'NOC', 'VZ'], value="AAPL"),
        gr.Slider(minimum=-1.0, maximum=1.0, value=0.4, label="Sentiment Score"),
        gr.Slider(minimum=0, maximum=350, value=50, label="Number of Tweets")
    ],
    outputs="text",
    title="Stock Price Prediction",
    description="Train the model on the selected stock symbol and predict the next day's closing price."
)
#sentiment tab
sentiment_tab = gr.Interface(
    fn=analyze_tweets,
    inputs=gr.Textbox(
        lines=5,
        placeholder="Enter multiple tweets separated by '|||'",
        label="Tweets"
    ),
    outputs="text",
    title="Tweet Sentiment Analysis",
    description="Enter multiple tweets separated by '|||', and get the average compound sentiment score."
)

#tabbed interface 
app = gr.TabbedInterface([stock_price_tab, sentiment_tab], tab_names=["Stock Price Prediction", "Tweet Sentiment Analysis"])

app.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://07e57ed5f280ee256c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Epoch 1/10


c:\Users\jdhof\anaconda3\envs\dev\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


153/153 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0495 - val_loss: 0.0216
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0224 - val_loss: 0.0088
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0132 - val_loss: 0.0095
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0111 - val_loss: 0.0092
Epoch 5/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0094 - val_loss: 0.0097
Epoch 6/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0143 - val_loss: 0.0157
Epoch 7/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0119 - val_loss: 0.0079
Epoch 8/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0105 - val_loss: 0.0130
Epoch 9/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0120 - val_loss: 0.0080
Epoch 10/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0119 - val_loss: 0.0061
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 314ms/step


[*********************100%%**********************]  1 of 1 completed

...


c:\Users\jdhof\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\jdhof\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\jdhof\anaconda3\envs\dev\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0455 - val_loss: 0.0097
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0151 - val_loss: 0.0101
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0115 - val_loss: 0.0086
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0076 - val_loss: 0.0081
Epoch 5/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0074 - val_loss: 0.0056
Epoch 6/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0106 - val_loss: 0.0059
Epoch 7/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0081 - val_loss: 0.0053
Epoch 8/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0074 - val_loss: 0.0071
Epoch 9/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0072 - val_loss: 0.0122
Epoch 10/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0094 - val_loss: 0.0053
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step


[*********************100%%**********************]  1 of 1 completed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step



c:\Users\jdhof\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\jdhof\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
